# DB에서 상장 기업 정보를 받아 네이버 증권에서 주가 수집하기

In [3]:
import os
import pandas as pd
from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
import pymysql
import time
import requests
from bs4 import BeautifulSoup as bs

In [2]:
pymysql.install_as_MySQLdb()
load_dotenv(dotenv_path=".env_db")

True

In [3]:
engine = create_engine(f'{os.getenv("db")}+{os.getenv("dbtype")}://{os.getenv("id")}:{os.getenv("pw")}@{os.getenv("host")}/{os.getenv("database")}')
conn = engine.connect()
data = pd.read_sql('2024_07_29_stock_company_info',con=conn)
conn.close()

In [8]:
for code in data['종목코드'][:1]:
    url=f"https://finance.naver.com/item/main.naver?code={code}"
    print(url)

https://finance.naver.com/item/main.naver?code=062040


In [14]:
r=requests.get(url)
    
soup=bs(r.text,'lxml')

In [16]:
#회사명
soup.select_one('h2').text

'산일전기'

In [17]:
#종목코드
soup.select_one('.code').text

'062040'

In [46]:
#현재가
soup.select_one('div > p.no_today .blind').text


'46,000'

In [50]:
#변동금액
soup.select_one('div > p.no_exday > em:nth-child(2) .blind').text


'11,000'

In [135]:
#변동률
soup.select_one('div > p.no_exday > em:nth-child(4) .blind').text



'31.43'

In [73]:
#전일
soup.select_one('.first em .blind').text

'35,000'

In [109]:
#고가
soup.select_one('td .no_up .blind').text

'49,200'

In [115]:
#거래량
soup.select_one('tr > td:nth-child(3) .blind').text

'15,138,846'

In [129]:
soup.select_one('#chart_area > div.rate_info > table > tbody > tr:nth-child(2)')

# 수집과 동시에 DB에 저장하기 + 예외처리 하기

In [7]:
today = str(date.today()).replace("-","_")
today[:7]

'2024_07'

In [8]:
def str2int(x):
    x = int(x.replace(",", ""))
    return x

In [ ]:
# DB에 접속해서 저장하는 함수
def stock_info_to_db(idx, code, df):
    from sqlalchemy import create_engine
    import pymysql
    from dotenv import load_dotenv
    from datetime import date
    pymysql.install_as_MySQLdb()
    load_dotenv(dotenv_path=".env_db")
    today = str(date.today()).replace("-","_")
    
    engine = create_engine(f"{os.getenv('db')}+{os.getenv('dbtype')}://{os.getenv('id')}:{os.getenv('pw')}@{os.getenv('host')}/{os.getenv('database')}")
    conn = engine.connect()
    df.to_sql(f"{today[:7]}_stock_price_info", con=conn, if_exists='append', index=False)
    conn.close()
    
    return print(f"{today}, {idx}, {code}, {'저장완료':<30s}", end="\r")
    

In [ ]:
errors = {}
for idx, (company, code) in enumerate(zip(data['회사명'], data['종목코드'])):
    stock_price_detail = {}
    url = f"https://finance.naver.com/item/main.naver?code={code}" 
    try:
        r = requests.get(url)
        print(r.status_code, f"{idx+1}/{len(data['종목코드'])} 수집중                    ", end="\r")
        soup = bs(r.text, 'lxml')
        # 가격
        price = int((soup.select_one(".today").text).strip("\n").split("\n")[1].replace(",", ""))
        # 변동금액
        price_chage = int((soup.select_one(".today").text).strip("\n").split("\n")[9].replace(",", ""))
        # 변화율
        rate_of_chage = float(((soup.select_one(".today").text).strip("\n").split("\n")[13]+(soup.select_one(".today").text).strip("\n").split("\n")[15]).replace("%",""))
        stock_price_detail.setdefault('수집일',[]).append(str(date.today()))
        stock_price_detail.setdefault('회사명', []).append(company)
        stock_price_detail.setdefault('종목코드', []).append(code)
        stock_price_detail.setdefault('현재가', []).append(price)
        stock_price_detail.setdefault('변동금액', []).append(price_chage)
        stock_price_detail.setdefault('변화율', []).append(rate_of_chage)
        table = soup.select_one(".no_info")
        for tr in table.select("tr"):
            for td in tr.select('td'):
                stock_price_detail.setdefault(td.select_one('span').text, []).append(str2int(td.select_one("span.blind").text))
        df = pd.DataFrame(stock_price_detail)
        stock_info_to_db(idx, code, df)
        time.sleep(5)
    except Exception as e:
        print(e)
        print(r.status_code, f"{idx+1}/{len(data['종목코드'])} 수집중 에러", end="\r")
        errors.setdefault("에러", []).append(code)  # 에러난 코드들 모음
        
stock_price_detail